In [ ]:
!pip install gurobipy
!pip install datetime
import gurobipy as gp
import calendar
from datetime import timedelta, datetime

# Create a new model
m = gp.Model()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Tìm ngày thứ 7 đầu tiên của tháng

In [ ]:
def find_first_monday(year, month):
    d = datetime(year, int(month), 1)
    offset = 5-d.weekday() #weekday = 5 means saturday
    return int((d + timedelta(offset)).date().strftime("%d"))

year = 2023
month = 6
print(find_first_monday(year, month))

3


# Tính số ngày trong tháng

In [ ]:
so_nv_parttime = 10
so_nv_fulltime = 25

# tổng các loại nv

# so_nv_parttime = so_nv_thu_ngan_parttime + so_nv_order_parttime + so_nv_bung_be_parttime
so_nv_thu_ngan_parttime = 2
so_nv_order_parttime = 3
so_nv_bung_be_parttime = 5

# so_nv_fulltime = so_nv_thu_ngan_full_time + so_nv_order_full_time + so_nv_bung_be_full_time
so_nv_thu_ngan_full_time = 4
so_nv_order_full_time = 6
so_nv_bung_be_full_time = 15

# Số ngày trong tháng chỉ định
so_ngay_trong_thang = calendar.mdays[month]

# Số nhân viên tối thiểu mỗi ca làm việc:

# Ca trưa
so_nv_thu_ngan_toi_thieu_ca_trua = 2
so_nv_order_toi_thieu_ca_trua = 3
so_nv_bung_be_toi_thieu_ca_trua = 10

# Câu tối
so_nv_thu_ngan_toi_thieu_ca_toi = 4
so_nv_order_toi_thieu_ca_toi = 6
so_nv_bung_be_toi_thieu_ca_toi = 14

# Số ngày được nghỉ
so_ngay_duoc_nghi = 4

# Tạo biến

## Nhân viên part time

In [ ]:
# Tạo biến cho nv part time
X = []

for i in range(so_nv_parttime):
  X_i = []
  for j in range(so_ngay_trong_thang * 2):
    X_i.append(m.addVar(vtype='B', name=f'X_{i+1}_{j+1}'))
  X.append(X_i)

## Nhân viên full time

In [ ]:
# Tạo biến nv full time
Y = []

for i in range(so_nv_fulltime):
  Y_i = []
  for j in range(so_ngay_trong_thang):
    Y_i.append(m.addVar(vtype='B', name=f'Y_{i+1}_{j+1}'))
  Y.append(Y_i)

# Tạo hàm mục tiêu

In [ ]:
tong_ca_X = sum([sum(X_i) for X_i in X])
tong_ca_Y = sum([sum(Y_i) for Y_i in Y])

In [ ]:
m.setObjective(
  100 * (tong_ca_X) 
  + 250 * (tong_ca_Y), gp.GRB.MINIMIZE)

# Tạo ràng buộc (Add constraints)

## Nhân viên có thể nghỉ 4 ngày/tháng

### Part-time

In [ ]:
for nhan_vien in range(so_nv_parttime):
  m.addConstr(sum(X[nhan_vien]) >= so_ngay_trong_thang - so_ngay_duoc_nghi) 
  m.addConstr(sum(X[nhan_vien]) <= so_ngay_trong_thang - 3) 

### Full-time

In [ ]:
for nhan_vien in range(so_nv_fulltime):
  m.addConstr(sum(Y[nhan_vien]) >= so_ngay_trong_thang - so_ngay_duoc_nghi)
  m.addConstr(sum(Y[nhan_vien]) <= so_ngay_trong_thang - 3)

## Mỗi tuần nhân viên part-time chỉ cần làm 1 ca/ngày

In [ ]:
for ca in range(so_ngay_trong_thang * 2 - 1):
  if (ca % 2 == 0):
    for nhan_vien in range(so_nv_parttime):
      temp = 0
      temp += X[nhan_vien][ca]
      temp += X[nhan_vien][ca + 1]
      m.addConstr(temp <= 1)

## Điều kiện thứ 7, chủ nhật không được nghỉ

### Part-time

In [ ]:
thu_7_dau_tien = find_first_monday(year,month)

for nhan_vien in range(so_nv_parttime):
  ca = thu_7_dau_tien*2 + 1
  while ca <= so_ngay_trong_thang * 2:
    m.addConstr(X[nhan_vien][ca-1] + X[nhan_vien][ca] >= 1)
    m.addConstr(X[nhan_vien][ca + 1] + X[nhan_vien][ca + 2] >= 1)
    ca += 14

### Full-time

In [ ]:
for nhan_vien in range(so_nv_parttime):
  ngay = thu_7_dau_tien
  while ngay <= so_ngay_trong_thang:
    m.addConstr(Y[nhan_vien][ngay] == 1)
    m.addConstr(Y[nhan_vien][ngay + 1] == 1)
    ngay += 7

## Tổng nhân viên từng loại cho từng ca

### Thu ngân

In [ ]:
for ca in range(so_ngay_trong_thang * 2):
  day = ca // 2

  # Ca trưa
  temp = 0
  if ca % 2 == 0:
    for thu_ngan in range(so_nv_thu_ngan_parttime):
      temp += X[thu_ngan][ca]
    
    for thu_ngan in range(so_nv_thu_ngan_full_time):
      temp += Y[thu_ngan][day]
    m.addConstr(temp >= so_nv_thu_ngan_toi_thieu_ca_trua)

  # Ca tối
  else:
    for thu_ngan in range(so_nv_thu_ngan_parttime):
      temp += X[thu_ngan][ca]
    
    for thu_ngan in range(so_nv_thu_ngan_full_time):
      temp += Y[thu_ngan][day]

    m.addConstr(temp >= so_nv_thu_ngan_toi_thieu_ca_toi)

### Order

In [ ]:
nhan_vien_order_parttime = so_nv_thu_ngan_parttime + so_nv_order_parttime
nhan_vien_order_fulltime = so_nv_thu_ngan_full_time + so_nv_order_full_time

for ca in range(so_ngay_trong_thang * 2):
  temp = 0
  day = ca // 2

  # Ca trưa
  if ca % 2 == 0:
    for thu_ngan in range(so_nv_thu_ngan_parttime, nhan_vien_order_parttime):
      temp += X[thu_ngan][ca]
    
    for thu_ngan in range(so_nv_thu_ngan_full_time, nhan_vien_order_fulltime):
      temp += Y[thu_ngan][day]
    m.addConstr(temp >= so_nv_order_toi_thieu_ca_trua)

  # Ca tối
  else:
    for thu_ngan in range(so_nv_thu_ngan_parttime, nhan_vien_order_parttime):
      temp += X[thu_ngan][ca]
    
    for thu_ngan in range(so_nv_thu_ngan_full_time, nhan_vien_order_fulltime):
      temp += Y[thu_ngan][day]
    m.addConstr(temp >= so_nv_order_toi_thieu_ca_toi)

### Bưng bê

In [ ]:
for ca in range(so_ngay_trong_thang * 2):
  temp = 0
  day = ca // 2

  # Ca trưa
  if ca % 2 == 0:
    for thu_ngan in range(-so_nv_bung_be_parttime, 0):
      temp += X[thu_ngan][ca]
    
    for thu_ngan in range(-so_nv_bung_be_full_time, 0):
      temp += Y[thu_ngan][day]
    m.addConstr(temp >= so_nv_bung_be_toi_thieu_ca_trua)

  # Ca tối
  else:
    for thu_ngan in range(-so_nv_bung_be_parttime, 0):
      temp += X[thu_ngan][ca]
    
    for thu_ngan in range(-so_nv_bung_be_full_time, 0):
      temp += Y[thu_ngan][day]
    m.addConstr(temp >= so_nv_bung_be_toi_thieu_ca_toi)

## Thời gian nghỉ không quá sát nhau

### Full-time

In [ ]:
# Thời gian nghỉ cách nhau ít nhất 1 tuần
for day in range(so_ngay_trong_thang - 7 + 1):
  for nhan_vien in range(so_nv_fulltime):
    temp = 0
    for i in range(7):
      temp += Y[nhan_vien][day + i]
    m.addConstr(temp == 6)

### Part-time

In [ ]:
# Thời gian nghỉ cách nhau ít nhất 1 tuần
for ca in range(so_ngay_trong_thang * 2 - 14 + 1):
  for nhan_vien in range(so_nv_parttime):
    temp = 0
    for index in range(14):
      temp += X[nhan_vien][ca + index]
    m.addConstr(temp == 6)

# Giải bài toán

In [ ]:
# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1780 rows, 1350 columns and 16420 nonzeros
Model fingerprint: 0x19ca1bba
Variable types: 0 continuous, 1350 integer (1350 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1735 rows and 1243 columns
Presolve time: 0.03s
Presolved: 45 rows, 107 columns, 319 nonzeros
Variable types: 0 continuous, 107 integer (105 binary)
Found heuristic solution: objective 188500.00000

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.01 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 188500 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.885000000000e+05, best bound 1.885000000000e+05, g

# Trả về kết quả dạng excel để người sử dụng dễ nhìn

In [ ]:
!pip install xlsxwriter
import xlsxwriter
 
workbook = xlsxwriter.Workbook(f'timetable{month}.xlsx')
 
# By default worksheet names in the spreadsheet will be
# Sheet1, Sheet2 etc., but we can also specify a name.
parttime_sheet = workbook.add_worksheet("Part time")
fulltime_sheet = workbook.add_worksheet("Full time")

work_format = workbook.add_format()
work_format.set_bg_color('#CCFF99')

title_format = workbook.add_format()
title_format.set_bold()
title_format.set_bg_color('#f8e08e')

for nhan_vien in range(so_nv_parttime):
  parttime_sheet.write(0, nhan_vien + 1, f'Nhân viên {nhan_vien + 1}', title_format)

  for ca in range(so_ngay_trong_thang*2):
    parttime_sheet.write(ca + 1, 0, f'Ca {ca+1}', title_format)
    is_work = X[nhan_vien][ca].X
    if is_work == 1:
      parttime_sheet.write(ca + 1, nhan_vien + 1, "Đi làm", work_format)
    else:
      parttime_sheet.write(ca + 1, nhan_vien + 1, "Nghỉ")


  for nhan_vien in range(so_nv_fulltime):
    fulltime_sheet.write(0, nhan_vien + 1, f'Nhân viên {nhan_vien + 1}', title_format)

    for ca in range(so_ngay_trong_thang):
      fulltime_sheet.write(ca + 1, 0, f'Ngày {ca+1}', title_format)
      is_work = Y[nhan_vien][ca].X
      if is_work == 1:
        fulltime_sheet.write(ca + 1, nhan_vien + 1, "Đi làm", work_format)
      else:
        fulltime_sheet.write(ca + 1, nhan_vien + 1, "Nghỉ")
      

parttime_sheet.autofit()
fulltime_sheet.autofit()

workbook.close()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
